## Autogen Sequential Chats Coding

*[Coding along with the Udemy online course AI Agents: Building Teams of LLM Agents that Work For You by Mohsen Hassan & Ilyass Tabiai]*

Agents can be used to onboard users into a live chat system to solve issues like you can see it on banking webiste or phone provider webistes.

For this example of a sequential chat we will assume that these agents are deployed by a phone provider company called "ACME". The agents are used to gather specifc information about a user that needs help, then to pass that information to a human that will directly have a clear understanding of who requires help, where they're from and what their problem is. The human should be able to immediately act on this information and help the user.

The accomplish this task 3 agents are needed:

An __Onboarding Personal Information Agent__ whose goal it is to get the name and location of the customer.

An __Onboarding Issue Agent__ whose it is goal to determine what the issue of the customer is.

A __Customer Engagement Agent__ who will interact with the customer until a human agent is available for the customer.

An autogen example for Solving Multiple Tasks in a Sequence of Chats can be found at the __[Autogen Docs](https://microsoft.github.io/autogen/docs/notebooks/agentchat_multi_task_chats/)__: The notebook there "showcases how to use the new chat interface of conversational agents in AutoGen: initiate_chats, to conduct a series of tasks. This new interface allows one to pass multiple tasks and their corresponding dedicated agents. Once initiate_chats is invoked, the tasks will be solved sequentially, with the summaries from previous tasks provided to subsequent tasks as context, if the summary_method argument is specified."

In [17]:
from openai import OpenAI
import pandas as pd
from autogen import ConversableAgent

In [18]:
api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [19]:
# creating a Conversable Agent to accomlpish a simple task
# 1st thing to do is to create a LLM config that specifies which LLM we want to use
# model from the list of models provided by OpenAI https://platform.openai.com/docs/models/continuous-model-upgrades
llm_config = {
    "model": "gpt-4o-mini",
    # "model": "gpt-3.5-turbo",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


### Agents definition: (1) Onboarding Personal Information Agent

The main goal of the Onboarding Personal Information Agent is to get the name and location of the customer. 

In [20]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you work for a phone provider called ACME.
    Your job is to gather the customer's name and location.
    Do not ask for any other information, only ask about the customer's name and location.
    After the customer gives you their name and location, repeat them 
    and thank the user, and ask the user to answer with TERMINATE to move on to describing their issue.
    ''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

[autogen.oai.client: 10-02 14:56:32] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Agents definition: (2) Onboarding Issue Agent

The main goal of the Onboarding Issue Agent is to determine the issue the customer is facing with ACME's products.

In [21]:
onboarding_issue_agent = ConversableAgent(
    name="Onboarding_Issue_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you work for a phone provider called ACME,
    you are here to help new customers get started with our product.
    Your job is to gather the product the customer use and the issue they currently 
    have with the product,
    Do not ask for other information.
    After the customer describes their issue, repeat it and add
    "Please answer with 'TERMINATE' if I have correctly understood your issue." ''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

[autogen.oai.client: 10-02 14:56:36] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Agents definition: (3) Customer Engagement Agent

The main goal of the **Customer Engagement Agent** is to interact with the customer based on the previously gathered information until a human agent is available to solve the customer's issue.

In [22]:
customer_engagement_agent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message='''You are a helpful customer service agent.
    Your job is to gather customer's preferences on news topics.
    You are here to provide fun and useful information to the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 10-02 14:56:39] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Agents definition: (4) Customer Proxy Agent

The question now is how to get the user into this conversation to engage with the agents. The achieve this we define a fourth agent, the Customer Proxy Agent.

The __Customer Proxy Agent__ is a Conversable Agent that allows the human (the user, which is you or us) to play the role of the agent. It's not an LLM therefore `llm_config=False` and it has `human_input_mode="ALWAYS"` which means that you, the human, will be the customer through this agent.

In [23]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False, # NO llm config or model here; can't interact with llm
    code_execution_config=False, # code execution would be possible but we don't want it here
    human_input_mode="ALWAYS", # NO llm but ALWAYS human input
)

### Next Step: Chat orchestration - Creating the Chat Sequence

Orchastrating how the chat will happen means that we will define in which order agents will interact and who'll interact with who when. To define this, we will use a list, that will contain several elements, each one corresponding to a chat. The chats will then happen in that specific order.

In [24]:
# defining a `chat` object
chats = [] # This is going to be our list of chats

### Chat orchestration (1): Onboarding Agent with Customer

We will now define the first chat and add it to this list. 

The first chat will be between our first agent, the **Onboarding Personal Information Agent** and the **customer**, who is going to be us.

The first message will be sent by the **Onboarding Personal Information Agent** and will be:
> *Hello, I'm here to help you get started with our product. 
            Could you tell me your name?*

__Carrying data to the next chat:__

__Specify the summary format:__ In order to make the transition easier with the next agent, we are going to ask for a slightly different type of summary than we did before with this agent. We are going to request a summary generated by the LLM, but **we will specify that the summary should return the name and location of the customer in a JSON format:** `{'name': '', 'location': ''}`. This is a structured data format that can be easily read by another agent but **also by another app or protocol**. This shows how an LLM agent can be used to interact with other apps.

__The clear_history paramter:__ Since we only want to transfer name and location to the next chat and we specifically specified how we want to transfer this data, we are going to add a new parameter, the `clear_history` to `True` which means that no data other than the one specified in the summary will be sent to the next chat. If we set it to `False` the agent from the next chat will be aware about the previous exchange with the user. We'll use that later.

In [25]:
chats.append(
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you solve any issue you have with our products. "
            "Could you tell me your name?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
        "summary_prompt" : "Return the customer information "
                             "into a JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "clear_history" : True
    }
)

In [26]:
chats

[{'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1060e22a0>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1074f5f40>,
  'message': "Hello, I'm here to help you solve any issue you have with our products. Could you tell me your name?",
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return the customer information into a JSON object only: {'name': '', 'location': ''}"},
  'clear_history': True}]

### Chat orchestration (2): Onboarding Agent with Customer

__Defining the second chat and adding it to this list:__ 

The second chat will be between our second agent, the **Issue Agent** and the **customer**, who is going to be us again.

The second message will be sent by the **Onboarding Personal Information Agent** and will be:
> *Great! Could you tell me what issue you're currently having and with which product?*

__Summary:__ This time we're going to generate a summary, but we won't specify any format or specifc data that must be carried over because we do not know what the exchange will yield specifically. 

__Chat History:__ We are also going to specify that we want to transfer the chat history to the next chat/agent.

In [27]:
chats.append(
    {
        "sender": onboarding_issue_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what issue you're "
                "currently having and with which product?",
        "summary_method": "reflection_with_llm",
        "clear_history" : False # history will be kept and transmitted to the next agent
    }
)
# the summary is called `context` and will be transmitted from on chat to the next

In [28]:
chats

[{'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1060e22a0>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1074f5f40>,
  'message': "Hello, I'm here to help you solve any issue you have with our products. Could you tell me your name?",
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return the customer information into a JSON object only: {'name': '', 'location': ''}"},
  'clear_history': True},
 {'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x12009a000>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1074f5f40>,
  'message': "Great! Could you tell me what issue you're currently having and with which product?",
  'summary_method': 'reflection_with_llm',
  'clear_history': False}]

### Chat orchestration (3): Customer Engagement Agent - Entertaining the Customer

__Defining the third chat and adding it to the list:__

The third chat will be between our third agent, the **Customer Engagement Agent** and the **customer**, who is going to be us again.

The third message will be sent by the **Customer Engagement Agent** and will be:
> *Can you tell me more about how you use our products or some topics interesting for you?*

__Summary:__ This time we're going to generate a summary so that the human agent can get this information in an easy and quick way when they take over the conversation, but we won't specify any format or specifc data that must be carried over because we do not know what the exchange will yield specifically. 

In [29]:
chats.append(
        {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "While we're waiting for a human agent to take over and help you solve "
        "your issue, can you tell me more about how you use our products or some "
        "topics interesting for you?",
        "max_turns": 2,
        "summary_method": "reflection_with_llm",
    }
)

In [30]:
chats

[{'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1060e22a0>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1074f5f40>,
  'message': "Hello, I'm here to help you solve any issue you have with our products. Could you tell me your name?",
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return the customer information into a JSON object only: {'name': '', 'location': ''}"},
  'clear_history': True},
 {'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x12009a000>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1074f5f40>,
  'message': "Great! Could you tell me what issue you're currently having and with which product?",
  'summary_method': 'reflection_with_llm',
  'clear_history': False},
 {'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x1074f5f40>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x120a11100>,
  'message

### Initiate the sequential chat

Now that we finished orchestrating the chat, __we can get it started!__  
For this to work, **you**, the customer, will have to roleplay as a customer that currently have an issue with your phone provider. Let's say that your internet does not work, or that you want more bandwidth, or that you want some help to setup port forwarding to play a game with some friends or some other thing. Have fun doing some roleplay!  

In [31]:
from autogen import initiate_chats

In [32]:
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Hello, I'm here to help you solve any issue you have with our products. Could you tell me your name?

--------------------------------------------------------------------------------


/Users/juergenkober/Projects/Python+Rust/_github-repos/working-with-financial-data/py-building-llm-agent-teams/env/lib/python3.12/site-packages/autogen/agentchat/chat.py:53: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to Onboarding_Personal_Information_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  John


customer_proxy_agent (to Onboarding_Personal_Information_Agent):

John

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Thank you, John! Can you please tell me your location?

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to Onboarding_Personal_Information_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Texas


...

customer_proxy_agent (to Onboarding_Issue_Agent):

I'm having a problem with my old Samsung that only runs Android 10

--------------------------------------------------------------------------------
Onboarding_Issue_Agent (to customer_proxy_agent):

You are having a problem with your old Samsung that only runs Android 10. Please answer with 'TERMINATE' if I have correctly understood your issue.

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to Onboarding_Issue_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  terminate


customer_proxy_agent (to Onboarding_Issue_Agent):

terminate

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to Customer_Engagement_Agent):

While we're waiting for a human agent to take over and help you solve your issue, can you tell me more about how you use our products or some topics interesting for you?
Context: 
```json
{
  "name": "John",
  "location": "Texas"
}
```
The customer, John from Texas, is experiencing an issue with his old Samsung device that runs on Android 10. He confirmed the understanding of his issue by responding with 'terminate.'

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to Customer_Engagement_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  how about using generative ai on my old samsung?


customer_proxy_agent (to Customer_Engagement_Agent):

how about using generative ai on my old samsung?

--------------------------------------------------------------------------------
Customer_Engagement_Agent (to customer_proxy_agent):

Hey John! That's an interesting question! While your old Samsung device running Android 10 may not be equipped with the latest hardware or software updates, you can still explore some generative AI tools. Here are a couple of fun ways you might engage with AI:

1. **Text Generation Apps**: There are apps that you can install which utilize generative AI to help with writing, like drafting stories or creating fun poems. Just check if they're compatible with Android 10.

2. **Chatbots**: You could try chatbots that offer entertainment or engage in conversation. It can be fun to see how well they handle jokes or creative prompts!

3. **Image Generators**: Some nice apps allow you to create art based on text prompts. They might have light versions or onlin

The last chat gets terminated automatically because we specified a `max_turns` settings for each chat.

Now once the human agent is ready to take over the conversation, we can provide them with __a summary of all the information necessary__ so that they can immediately get started with solving the issue of the customer using the following commads, as we previously explored:

In [33]:
import pprint

In [34]:
for chat_result in chat_results:
    #pprint.pprint(chat_result.chat_history) # We could also get the whole chat history with this command
    pprint.pprint(chat_result.summary)

'```json\n{\n  "name": "John",\n  "location": "Texas"\n}\n```'
('The customer, John from Texas, is experiencing an issue with his old Samsung '
 'device that runs on Android 10. He confirmed the understanding of his issue '
 "by responding with 'terminate.'")
('John is interested in using generative AI on his old Samsung device running '
 'Android 10. Possible applications include text generation apps, chatbots, '
 'and image generators. He can explore these tools while considering the '
 "device's limitations. The conversation highlights various fun and creative "
 'uses of generative AI that could suit his interests.')


The human can thus quickly learn about our customer, their name, location and issue and immediately get started on solving the issue.

In [35]:
# printing the whole chat history
for chat_result in chat_results:
    pprint.pprint(chat_result.chat_history)

[{'content': "Hello, I'm here to help you solve any issue you have with our "
             'products. Could you tell me your name?',
  'name': 'Onboarding_Personal_Information_Agent',
  'role': 'assistant'},
 {'content': 'John', 'name': 'customer_proxy_agent', 'role': 'user'},
 {'content': 'Thank you, John! Can you please tell me your location?',
  'name': 'Onboarding_Personal_Information_Agent',
  'role': 'assistant'},
 {'content': 'Texas', 'name': 'customer_proxy_agent', 'role': 'user'},
 {'content': 'Thank you, John from Texas! Please respond with "TERMINATE" to '
             'move on to describing your issue.',
  'name': 'Onboarding_Personal_Information_Agent',
  'role': 'assistant'},
 {'content': 'Terminate', 'name': 'customer_proxy_agent', 'role': 'user'}]
[{'content': "Great! Could you tell me what issue you're currently having and "
             'with which product?\n'
             'Context: \n'
             '```json\n'
             '{\n'
             '  "name": "John",\n'
   

## What is this for?

This example shows you how you can integrate LLMs into an app in order to accomplish complex interactions with humans and gather specific information through natural communication. This is very powerful and is not something that was easy to do it before having LLMs. By having agents focused on a single task, we're able to ensure that the LLM won't diverge and remain focused on a simple task, greatly increasing chances that it accurately accomplishes it.